In [1]:
import pandas as pd

from env import username, password, host
import wrangle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Acquire

In [3]:
url = f"mysql+pymysql://{username}:{password}@{host}/zillow"
    
query = """
            
SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, taxamount, assessmentyear, regionidcounty,'regionidzip', transactiondate
FROM properties_2017

LEFT JOIN propertylandusetype USING(propertylandusetypeid)

JOIN predictions_2017 USING(parcelid)

WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")
                              AND (transactiondate BETWEEN '2017-05-01' AND '2017-08-31');
                              
                              """
    
df = pd.read_sql(query, url)
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,assessmentyear,regionidcounty,regionidzip,transactiondate
0,3.0,2.0,1316.0,205123.0,2627.48,2016.0,3101.0,regionidzip,2017-07-21
1,3.0,2.0,1458.0,136104.0,2319.90,2016.0,3101.0,regionidzip,2017-06-23
2,2.0,1.0,1421.0,35606.0,543.69,2016.0,3101.0,regionidzip,2017-06-30
3,4.0,3.0,2541.0,880456.0,9819.72,2016.0,1286.0,regionidzip,2017-06-01
4,3.0,2.0,1491.0,107110.0,1399.27,2016.0,3101.0,regionidzip,2017-08-23
...,...,...,...,...,...,...,...,...,...
28119,6.0,2.0,2634.0,878190.0,12766.88,2016.0,3101.0,regionidzip,2017-08-31
28120,4.0,1.5,1187.0,465999.0,5552.68,2016.0,1286.0,regionidzip,2017-08-31
28121,5.0,3.0,3015.0,852813.0,11662.88,2016.0,1286.0,regionidzip,2017-08-31
28122,5.0,3.0,2992.0,1120000.0,13663.03,2016.0,3101.0,regionidzip,2017-08-31


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28124 entries, 0 to 28123
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    28124 non-null  float64
 1   bathroomcnt                   28124 non-null  float64
 2   calculatedfinishedsquarefeet  28077 non-null  float64
 3   taxvaluedollarcnt             28123 non-null  float64
 4   taxamount                     28123 non-null  float64
 5   assessmentyear                28124 non-null  float64
 6   regionidcounty                28124 non-null  float64
 7   regionidzip                   28124 non-null  object 
 8   transactiondate               28124 non-null  object 
dtypes: float64(7), object(2)
memory usage: 1.9+ MB


In [5]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,assessmentyear,regionidcounty,regionidzip,transactiondate
0,3.0,2.0,1316.0,205123.0,2627.48,2016.0,3101.0,regionidzip,2017-07-21
1,3.0,2.0,1458.0,136104.0,2319.90,2016.0,3101.0,regionidzip,2017-06-23
2,2.0,1.0,1421.0,35606.0,543.69,2016.0,3101.0,regionidzip,2017-06-30
3,4.0,3.0,2541.0,880456.0,9819.72,2016.0,1286.0,regionidzip,2017-06-01
4,3.0,2.0,1491.0,107110.0,1399.27,2016.0,3101.0,regionidzip,2017-08-23


In [6]:
cols = ['bedroomcnt','bathroomcnt','calculatedfinishedsquarefeet','taxvaluedollarcnt','taxamount','assessmentyear']
# impute cols using model
imputer = SimpleImputer(strategy='most_frequent')

imputer.fit(df[cols])

df[cols] = imputer.transform(train[cols])
    

NameError: name 'train' is not defined

In [ ]:
#n removing outliers
df = wrangle.remove_outliers(df, 1.5, ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'taxamount', 'assessmentyear', 'regionidcounty'])
df.head()

In [ ]:
# List of columns
cols = [col for col in df.columns if col not in ['fips', 'year_built']]

In [ ]:
# calculate the tax rate
df['tax_rate']= df['taxamount']/df['taxvaluedollarcnt']

In [ ]:
df.head()

In [ ]:
#narrow down the record to those in the "hot months" May-August 2017
# this df created so far is for answering the questions posed by team
# it will not be used to create models